In [31]:
# task 2.2
from datetime import datetime
import pandas as pd
#reading file

direction_1 = []
direction_2 = []
with open('ARRIVALS.TXT') as f :
    arrivals = [line.strip().split(',') for line in f.readlines()[1:]]
    for [seq, direction, code, dist, time] in arrivals:
        arr = [int(seq), direction, code, float(dist), datetime.strptime(time,'%H%M')]
        direction_1.append(arr) if direction=='1' else direction_2.append(arr)

print(f"{'Direction':^9} {'Start':<5} {'End':<5} {'Total':<6}")

min_d1 = min(direction_1, key=lambda record : record[4])[4]
max_d1 = max(direction_1, key=lambda record : record[4])[4]
hours, minutes = (max_d1-min_d1).seconds//3600, ((max_d1-min_d1).seconds//60)%60 
print(f"{'1':^9} {min_d1.strftime('%H%M') :<5} {max_d1.strftime('%H%M') :<5} {hours}H,{minutes}m")

min_d2 = min(direction_2, key=lambda record : record[4])[4]
max_d2 = max(direction_2, key=lambda record : record[4])[4]
hours2, minutes2 = (max_d2-min_d2).seconds//3600, ((max_d2-min_d2).seconds//60)%60 
print(f"{'1':^9} {min_d2.strftime('%H%M') :<5} {max_d2.strftime('%H%M') :<5} {hours2}H,{minutes2}m")

Direction Start End   Total 
    1     0500  0627  1H,27m
    1     0600  0730  1H,30m


In [32]:
# task 2.3

def time_diff(arr : list):
    differences = []
    for i in range(len(arr)-1):
        time_difference = (arr[i+1][4]-arr[i][4]).seconds//60
        if time_difference > 2 :
            bus_stop1, bus_stop2 = arr[i][2], arr[i+1][2]
            differences.append((bus_stop1, bus_stop2, time_difference))
    return differences

def display(direction, arr:list):
    print(f'Direction {direction}')
    for record in arr :
        print(f"{record[0]} to {record[1]}: {record[2]}m")
direction_1.sort(key=lambda record : record[0])
direction_2.sort(key=lambda record : record[0])

time_diff_d1 = time_diff(direction_1)
time_diff_d2 = time_diff(direction_2)

display(1, time_diff_d1)
display(2, time_diff_d2)

Direction 1
76069 to 96289: 4m
80219 to 80169: 3m
02119 to 03019: 3m
03217 to 05649: 3m
Direction 2
14089 to 14069: 3m
80161 to 80211: 4m
96281 to 76061: 4m


In [33]:
# task 2.4
def max_dist_diff(arr:list):
    differences=[]
    for i in range(len(arr)-1):
        differences.append(arr[i+1][3]-arr[i][3])
    return max(differences)

max_dist_d1 = max_dist_diff(direction_1)
max_dist_d2 = max_dist_diff(direction_2)

max_dist = [max_dist_d1, max_dist_d2]
for i in range(1, 3):
    print(f"Direction {i}\nlongest distance between two adjacent bus stops is {max_dist[i-1]:.2f} km")

Direction 1
longest distance between two adjacent bus stops is 1.40 km
Direction 2
longest distance between two adjacent bus stops is 1.60 km


In [34]:
# task 2.5
def min_dist_btwn_bus(arr1 : list, arr2 : list):
    distances = []
    for i in range(len(arr1)):
        for j in range(len(arr2)):
            bus_stop1, bus_stop2 = arr1[i][2], arr2[j][2]
            time1, time2 = arr1[i][4], arr2[j][4]
            distance = abs(31.8-arr1[i][3]-arr2[j][3])
            if bus_stop1 != bus_stop2 and time1==time2:
                distances.append((bus_stop1, bus_stop2, time1, distance))
    return min(distances, key=lambda x : x[-1])

bus_stop1,bus_stop2, arrival_time = min_dist_btwn_bus(direction_1,direction_2)[:-1]

print(f"The time in which the two buses is closest to each other is at {arrival_time.strftime('%H%M')} at bus stops {bus_stop1} and {bus_stop2}")

The time in which the two buses is closest to each other is at 0615 at bus stops 15161 and 15159


In [43]:
#task 2.5 with sql

import pandas as pd
import sqlite3

direction_1 = pd.DataFrame(direction_1, columns='StopSequence,Direction,BusStopCode,Distance,ArrivalTime'.split(',')).drop('Direction', axis=1)
direction_2 = pd.DataFrame(direction_2, columns='StopSequence,Direction,BusStopCode,Distance,ArrivalTime'.split(',')).drop('Direction', axis=1)

con = sqlite3.connect('task2.db')
direction_1.to_sql('Direction1', con, if_exists='replace', index=False)
direction_2.to_sql('Direction2', con, if_exists='replace', index=False)

cursor = con.cursor()
cursor.execute("""
            SELECT * FROM  Direction1 CROSS JOIN Direction2 
            WHERE Direction1.BusStopCode != Direction2.BusStopCode AND
            Direction1.ArrivalTime = Direction2.ArrivalTime 
            ORDER BY ABS(31.8 - Direction1.Distance - Direction2.Distance) ASC""")

results = cursor.fetchone()
cursor.close()
con.close()

busstop1, busstop2, time_closest = results[1], results[5], datetime.strptime(results[3], '%Y-%m-%d %H:%M:%S').strftime('%H%M')
print(f"The time in which the two buses is closest to each other is at {time_closest} at bus stops {busstop1} and {busstop2}")

The time in which the two buses is closest to each other is at 0615 at bus stops 15161 and 15159
